# Multi-query retriever

## Librerías

In [1]:
import logging
from typing import Any

from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.retrievers import MultiQueryRetriever
from langchain.vectorstores import Chroma
from pydantic import BaseModel, Field

from src.langchain_docs_loader import load_langchain_docs_splitted

load_dotenv()

True

## Carga de datos

In [2]:
docs = load_langchain_docs_splitted()
len(docs)

2692

## Preparación de vectorstore

In [3]:
embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-vwqjdaXGZeEg6mWAVSflJXD9 on tokens per min. Limit: 1000000 / min. Current: 722919 / min. Contact us through our help center at help.openai.com if you continue to have issues..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-vwqjdaXGZeEg6mWAVSflJXD9 on tokens per min. Limit: 1000000 / min. Current: 614137 / min. Contact us through our help center at help.openai.com if you continue to have issues..


## Preparación de retriever

In [4]:
llm = ChatOpenAI(temperature=0)
retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=llm,
)

In [5]:
logging.basicConfig()
logging.getLogger("langchain.retrievers.multy_query").setLevel(logging.INFO)

##  Prueba de retriever

In [6]:
retriever.get_relevant_documents(
    "How to create a retriever with langchain expression language?"
)

[Document(page_content='# Code understanding\n\nOverview\n\nLangChain is a useful tool designed to parse GitHub code repositories. By leveraging VectorStores, Conversational RetrieverChain, and GPT-4, it can answer questions in the context of an entire GitHub repository or generate new code. This documentation page outlines the essential components of the system and guides using LangChain for better code comprehension, contextual question answering, and code generation in GitHub repositories.\n\n## Conversational Retriever Chain\u200b\n\nConversational RetrieverChain is a retrieval-focused system that interacts with the data stored in a VectorStore. Utilizing advanced techniques, like context-aware filtering and ranking, it retrieves the most relevant code snippets and information for a given user query. Conversational RetrieverChain is engineered to deliver high-quality, pertinent results while considering conversation history and context.\n\nLangChain Workflow for Code Understanding 

## Generación de preguntas alternativas de forma personalizada

### Definición de esquema de salida de preguntas

In [7]:
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: list[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser[Any]):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().splitlines()
        return LineList(lines=lines)

### Creación de `prompt` personalizado

In [8]:
prompt = PromptTemplate.from_template(
    """You are an AI language assistant well versed in the Langchain Documentation.
Your more precise task is to generate five different versions of the given question to retrieve relevant documents from a vector database.
By generating multiple perspectives on the question, your goal is to overcome some of the limitations of the distance-based similarity search.

Provide these alternative questions separed by newlines.

Original question: {question}
New questions:"""
)

llm_chain = LLMChain(
    llm=ChatOpenAI(temperature=0),
    prompt=prompt,
    output_parser=LineListOutputParser(),
)

# In language expression language, you could create the chain with:
# llm_chain = prompt | llm | LineListOutputParser()

### Use de cadena de generación de preguntas personalizada

In [9]:
llm_chain.invoke(
    {"question": "How to create a retriever with langchain expression language?"}
)

{'question': 'How to create a retriever with language expression language?',
 'text': LineList(lines=['1. What are the steps to build a retriever using language expression language?', '2. Can you explain the process of creating a retriever using language expression language?', '3. What is the procedure for constructing a retriever with language expression language?', '4. How can I develop a retriever using language expression language?', '5. Are there any guidelines for building a retriever with language expression language?'])}

### Integración de cadena de generación de preguntas personalizada en `retriever`

In [10]:
retriever = MultiQueryRetriever(
    retriever=vectorstore.as_retriever(),
    llm_chain=llm_chain,
    parser_key="lines",
)

### Uso de `retriever` con cadena de generación de preguntas personalizada

In [11]:
retriever.get_relevant_documents(
    "How to create a retriever with lagnchain expression language?"
)

[Document(page_content='# Code understanding\n\nOverview\n\nLangChain is a useful tool designed to parse GitHub code repositories. By leveraging VectorStores, Conversational RetrieverChain, and GPT-4, it can answer questions in the context of an entire GitHub repository or generate new code. This documentation page outlines the essential components of the system and guides using LangChain for better code comprehension, contextual question answering, and code generation in GitHub repositories.\n\n## Conversational Retriever Chain\u200b\n\nConversational RetrieverChain is a retrieval-focused system that interacts with the data stored in a VectorStore. Utilizing advanced techniques, like context-aware filtering and ranking, it retrieves the most relevant code snippets and information for a given user query. Conversational RetrieverChain is engineered to deliver high-quality, pertinent results while considering conversation history and context.\n\nLangChain Workflow for Code Understanding 